> # Import Dependencies



In [ ]:
import numpy as np
import pandas as pd
import re, requests
from IPython.display import clear_output
from bs4 import BeautifulSoup
import ast
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # represent string in form of numbet
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity # similarity between two things
import string,time
from nltk.corpus import stopwords
import nltk

> # Read Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
credits=pd.read_csv("/content/drive/MyDrive/m/credits.csv")
keywords=pd.read_csv("/content/drive/MyDrive/m/keywords.csv")
links=pd.read_csv("/content/drive/MyDrive/m/links.csv")
links_small=pd.read_csv("/content/drive/MyDrive/m/links_small.csv")
movies_metadata=pd.read_csv("/content/drive/MyDrive/m/movies_metadata.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


> # Preprocessing

>> ## First step : Basics Preprocessing

In [ ]:
print("Movies_meta_data shape : ", movies_metadata.shape)
print("Creadits shape : ", credits.shape)
print("Movies metadata columns : ", movies_metadata.columns)
print("Credits scolumns : ", credits.columns)

Movies_meta_data shape :  (45466, 24)
Creadits shape :  (45476, 3)
Movies metadata columns :  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Credits scolumns :  Index(['cast', 'crew', 'id'], dtype='object')


In [ ]:
# df = movies_metadata.merge(credits,on="id")

In [ ]:
movies_metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [ ]:
credits.dtypes

cast    object
crew    object
id       int64
dtype: object

In [ ]:
movies_metadata['id'] = pd.to_numeric(movies_metadata["id"],errors="coerce")

In [ ]:
movies_metadata.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           3
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [ ]:
movies_metadata.dropna(subset=["id"],inplace=True)

In [ ]:
movies_metadata.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   3
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      3
runtime                    260
spoken_languages             3
status                      84
tagline                  25051
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

In [ ]:
movies_metadata["id"] = movies_metadata["id"].astype(int, errors='ignore')

In [ ]:
movies_metadata['id']

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45463, dtype: int64

In [ ]:
df = movies_metadata.merge(credits,on="id")

In [ ]:
df.shape

(45538, 26)

In [ ]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

>>>>> **Applying Special Case :** Taking Only Some movies (Testing with : 35000)

In [ ]:
df.describe()

,id,revenue,runtime,vote_average,vote_count
count,45538.000000,4.553500e+04,45278.000000,45535.000000,45535.000000
mean,108316.948527,1.119860e+07,94.128517,5.618232,109.791128
std,112403.610181,6.428227e+07,38.396184,1.924203,490.920864
min,2.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,26411.500000,0.000000e+00,85.000000,5.000000,3.000000
50%,60012.500000,0.000000e+00,95.000000,6.000000,10.000000
75%,157170.750000,0.000000e+00,107.000000,6.800000,34.000000
max,469172.000000,2.787965e+09,1256.000000,10.000000,14075.000000


In [ ]:
35000*35000*8/1024/1024/1024

9.12696123123169

In [ ]:
df[df['revenue']==0.0].shape # not meaningful

(38107, 26)

In [ ]:
df[df['vote_average']==0.0].shape

(3004, 26)

In [ ]:
df[df['vote_count']==0.0].shape

(2905, 26)

In [ ]:
df = df[df['vote_count']>0.0]

In [ ]:
df.shape

(42630, 26)

In [ ]:
df = df[df['vote_average']>0.0]

In [ ]:
# df.isnull().sum()

In [ ]:
df = df.reset_index()

In [ ]:
df.columns

Index(['index', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [ ]:
df[df['runtime']>=95].shape

(21943, 27)

In [ ]:
df = df[df['runtime']>=95]

In [ ]:
df = df.reset_index()

>>>> **Ending Above Part**

In [ ]:
df = df[['genres', 'id','imdb_id','overview','poster_path','tagline', 'title','cast', 'crew']]

In [ ]:
df.shape

(21943, 9)

In [ ]:
df.isnull().sum()

genres             0
id                 0
imdb_id            4
overview         143
poster_path       24
tagline        10780
title              0
cast               0
crew               0
dtype: int64

In [ ]:
df.columns

Index(['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'tagline',
       'title', 'cast', 'crew'],
      dtype='object')

In [ ]:
(23000*23000*8)/1024/1024/1024

3.941357135772705

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview', 'poster_path','title', 'cast', 'crew']]

In [ ]:
df.isnull().sum()

genres           0
id               0
imdb_id          4
overview       143
poster_path     24
title            0
cast             0
crew             0
dtype: int64

In [ ]:
df.shape

(21943, 8)

>> ## Filling IMDB_id null values

In [ ]:
df = df.reset_index()
df['imdb_id'] = df['imdb_id'].fillna("be-coll-finding")

In [ ]:
null_values_index = df.index[df['imdb_id'] == "be-coll-finding"].tolist()

In [ ]:
null_values_index

[5970, 10808, 11744, 13632]

In [ ]:
# df.iloc[37034]

In [ ]:
api_key = "3222076ed226933d3e1302ce9f3b73dc"
movie_id = df.iloc[9001].id
url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US"
response = requests.get(url)
print(response.status_code)
doc = response.json()

200


In [ ]:
doc['imdb_id']

'tt0103626'

In [ ]:
found_null_values = []

for index in null_values_index:
    api_key = "3222076ed226933d3e1302ce9f3b73dc"
    movie_id = df.iloc[index].id
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US"
    response = requests.get(url)
    # print(response.status_code)
    if response.status_code == 200:
        doc = response.json()
        if doc['imdb_id'] == "":
            found_null_values.append(None)
        else:
            found_null_values.append(doc['imdb_id'])
    else: # unsuccess status code / unable to fetch
        found_null_values.append(None)

In [ ]:
found_null_values

['tt0101704', None, None, None]

In [ ]:
df.loc[df.imdb_id=="be-coll-finding", 'imdb_id'] = found_null_values

In [ ]:
df.isnull().sum()

index            0
genres           0
id               0
imdb_id          3
overview       143
poster_path     24
title            0
cast             0
crew             0
dtype: int64

In [ ]:
df.dropna(subset=["imdb_id"],inplace=True)

In [ ]:
df.isnull().sum()

index            0
genres           0
id               0
imdb_id          0
overview       143
poster_path     23
title            0
cast             0
crew             0
dtype: int64

>> ## Filling overview null values

In [ ]:
from IPython.display import clear_output
df = df.reset_index()
df['overview'] = df['overview'].fillna("be-coll-finding")
null_values_index = df.index[df['overview'] == "be-coll-finding"].tolist()

found_null_values = []
count = 1

for index in null_values_index:
    print(count)
    count += 1
    api_key = "3222076ed226933d3e1302ce9f3b73dc"
    movie_id = df.iloc[index].id
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US"
    response = requests.get(url)
    # print(response.status_code)
    if response.status_code == 200:
        doc = response.json()
        if doc['overview'] == "":
            found_null_values.append(None)
        else:
            found_null_values.append(doc['overview'])
    else: # unsuccess status code / unable to fetch
        found_null_values.append(None)
    clear_output(wait = True)

df.loc[df.overview == "be-coll-finding", 'overview'] = found_null_values

143


In [ ]:
df.isnull().sum()

level_0         0
index           0
genres          0
id              0
imdb_id         0
overview       26
poster_path    23
title           0
cast            0
crew            0
dtype: int64

In [ ]:
df.dropna(subset=["overview"],inplace=True)

In [ ]:
df.isnull().sum()

level_0         0
index           0
genres          0
id              0
imdb_id         0
overview        0
poster_path    22
title           0
cast            0
crew            0
dtype: int64

In [ ]:
df.isnull().sum()

level_0         0
index           0
genres          0
id              0
imdb_id         0
overview        0
poster_path    22
title           0
cast            0
crew            0
dtype: int64

>> ## Filling poster_path null values

In [ ]:
df.columns

Index(['level_0', 'index', 'genres', 'id', 'imdb_id', 'overview',
       'poster_path', 'title', 'cast', 'crew'],
      dtype='object')

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview',
       'poster_path', 'title', 'cast', 'crew']]

In [ ]:
df = df.reset_index()

In [ ]:
from IPython.display import clear_output

df['poster_path'] = df['poster_path'].fillna("be-coll-finding")
null_values_index = df.index[df['poster_path'] == "be-coll-finding"].tolist()

found_null_values = []
count = 1

for index in null_values_index:
    print(count)
    count += 1
    api_key = "3222076ed226933d3e1302ce9f3b73dc"
    movie_id = df.iloc[index].id
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US"
    response = requests.get(url)
    # print(response.status_code)
    if response.status_code == 200:
        doc = response.json()
        if doc['poster_path'] == "":
            found_null_values.append(None)
        else:
            found_null_values.append(doc['poster_path'])
    else: # unsuccess status code / unable to fetch
        found_null_values.append(None)
    clear_output(wait = True)

df.loc[df.poster_path == "be-coll-finding", 'poster_path'] = found_null_values

22


In [ ]:
df.isnull().sum()

index          0
genres         0
id             0
imdb_id        0
overview       0
poster_path    6
title          0
cast           0
crew           0
dtype: int64

In [ ]:
df.dropna(subset=["poster_path","title"],inplace=True)

In [ ]:
df.isnull().sum()

index          0
genres         0
id             0
imdb_id        0
overview       0
poster_path    0
title          0
cast           0
crew           0
dtype: int64

In [ ]:
df.shape

(21908, 9)

In [ ]:
df = df.reset_index()

>> ## genres, cast, crew - '[]' null vector data remove

In [ ]:
# %load_ext google.colab.data_table
# from google.colab import data_table
# data_table.enable_dataframe_formatter()
# data_table.disable_dataframe_formatter()

In [ ]:
df.columns

Index(['level_0', 'index', 'genres', 'id', 'imdb_id', 'overview',
       'poster_path', 'title', 'cast', 'crew'],
      dtype='object')

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview',
       'poster_path', 'title', 'cast', 'crew']]

In [ ]:
df.index

RangeIndex(start=0, stop=21908, step=1)

In [ ]:
df.shape[0]

21908

In [ ]:
df['genres'][0]

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [ ]:
null_index = []
for i in range(df.shape[0]):
    if df['genres'][i] == '[]':
        null_index.append(i)

In [ ]:
len(null_index)

552

In [ ]:
df = df.drop(df.index[null_index])

In [ ]:
df.shape

(21356, 8)

In [ ]:
df = df.reset_index()

In [ ]:
df[:2]

,index,genres,id,imdb_id,overview,poster_path,title,cast,crew
0,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
1,1,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [ ]:
df.columns

Index(['index', 'genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew'],
      dtype='object')

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew']]

In [ ]:
df[:2]

,genres,id,imdb_id,overview,poster_path,title,cast,crew
0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
1,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [ ]:
null_index = []
for i in range(df.shape[0]):
    if df['cast'][i] == '[]':
        null_index.append(i)

In [ ]:
len(null_index)

266

In [ ]:
df = df.drop(df.index[null_index])

In [ ]:
df.shape

(21090, 8)

In [ ]:
df = df.reset_index()

In [ ]:
df.columns

Index(['index', 'genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew'],
      dtype='object')

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew']]

In [ ]:
df.shape

(21090, 8)

In [ ]:
null_index = []
for i in range(df.shape[0]):
    if df['crew'][i] == '[]':
        null_index.append(i)

In [ ]:
len(null_index)

73

In [ ]:
df = df.drop(df.index[null_index])

In [ ]:
df = df.reset_index()

In [ ]:
df.columns

Index(['index', 'genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew'],
      dtype='object')

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew']]

>> ## Preprocessing on Cast, Crew

In [ ]:
def fetch_casts(list_of_dict):
    list_of_dict = ast.literal_eval(list_of_dict)
    lst = []
    for dict_ in list_of_dict[:min(6,len(list_of_dict))]:
        lst.append(dict_['name'])
    return lst

In [ ]:
df['cast'] = df['cast'].apply(fetch_casts)

In [ ]:
df['cast'][0]

['Robin Williams',
 'Jonathan Hyde',
 'Kirsten Dunst',
 'Bradley Pierce',
 'Bonnie Hunt',
 'Bebe Neuwirth']

In [ ]:
df['genres'] = df['genres'].apply(fetch_casts)

In [ ]:
def get_director(list_of_dict):
    list_of_dict = ast.literal_eval(list_of_dict)
    for dict_ in list_of_dict:
        if dict_['job'] == 'Director':
            return dict_['name']

In [ ]:
df['director'] = df['crew'].apply(get_director)

In [ ]:
df.isnull().sum()

genres          0
id              0
imdb_id         0
overview        0
poster_path     0
title           0
cast            0
crew            0
director       19
dtype: int64

In [ ]:
df.dropna(subset = ['director'],inplace=True)

In [ ]:
df.isnull().sum()

genres         0
id             0
imdb_id        0
overview       0
poster_path    0
title          0
cast           0
crew           0
director       0
dtype: int64

In [ ]:
df = df.reset_index()

In [ ]:
df.columns

Index(['index', 'genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
df = df[['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title',
       'cast','director']]

In [ ]:
df.columns

Index(['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title', 'cast',
       'director'],
      dtype='object')

In [ ]:
df.shape,df.index

((20998, 8), RangeIndex(start=0, stop=20998, step=1))

>> ## Taking Everything in one columns

In [ ]:
def list_of_string_To_string(lst):
    ans = ""
    for i in lst:
        ans += i + " "
    return ans

In [ ]:
df['cast'] = df['cast'].apply(list_of_string_To_string)

In [ ]:
df['genres'] = df['genres'].apply(list_of_string_To_string)

In [ ]:
count = 0
for i in df['overview']:
    if i=="":count += 1
print(count)

0


In [ ]:
all = []
for i in range(df.shape[0]):
  curr = df.iloc[i]
  all.append(curr['genres'] + " " + curr['cast'] + " " + curr['director'] + " " + curr['overview'])

In [ ]:
df.columns

Index(['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title', 'cast',
       'director'],
      dtype='object')

In [ ]:
df.shape, df.index

((20998, 8), RangeIndex(start=0, stop=20998, step=1))

In [ ]:
df['all'] = all

In [ ]:
df[:2]

,genres,id,imdb_id,overview,poster_path,title,cast,director,all
0,Adventure Fantasy Family,8844,tt0113497,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,Adventure Fantasy Family Robin Williams Jonat...
1,Romance Comedy,15602,tt0113228,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,Romance Comedy Walter Matthau Jack Lemmon Ann...


In [ ]:
df.columns

Index(['genres', 'id', 'imdb_id', 'overview', 'poster_path', 'title', 'cast',
       'director', 'all'],
      dtype='object')

In [ ]:
df = df[['id', 'imdb_id', 'poster_path', 'title','all']]

>> ## Before Applying : ML Algo

>>> ### Step 1 : Converting to lowercase

In [ ]:
df['all'] = df['all'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['all']

0        adventure fantasy family  robin williams jonat...
1        romance comedy  walter matthau jack lemmon ann...
2        comedy drama romance  whitney houston angela b...
3        comedy  steve martin diane keaton martin short...
4        action crime drama thriller  al pacino robert ...
                               ...                        
20993    drama  martijn fischer hadewych minis fedja va...
20994    comedy  hilary duff christy carlson romano gar...
20995    crime drama thriller  sridevi kapoor sajal ali...
20996    drama action romance  patrick bergin uma thurm...
20997    drama  angel aquino perry dizon hazel orencio ...
Name: all, Length: 20998, dtype: object

>>> ### Step 2: Removing HTML Tag

In [ ]:
def remove_HTML_Tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

In [ ]:
df['all'] = df['all'].apply(remove_HTML_Tags)

>>> ### Step 3: Remove URLs

In [ ]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [ ]:
df['all'] = df['all'].apply(remove_urls)

>>> ### Step 4: Remove Punctuations

In [ ]:
def remove_punctuations(text):
    exclude = string.punctuation
    for ch in exclude:
        text = text.replace(ch,'')
    return text

In [ ]:
df['all'] = df['all'].apply(remove_punctuations)

>>> ### Step 5: Remove StopWords

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_stopWords(text):
    stopWords = stopwords.words('english')
    lst = []
    for word in text.split():
        if word not in stopWords:
            lst.append(word)
    return " ".join(lst)

In [ ]:
df['all'] = df['all'].apply(remove_stopWords)

In [ ]:
df['all'][0]

'adventure fantasy family robin williams jonathan hyde kirsten dunst bradley pierce bonnie hunt bebe neuwirth joe johnston siblings judy peter discover enchanted board game opens door magical world unwittingly invite alan adult whos trapped inside game 26 years living room alans hope freedom finish game proves risky three find running giant rhinoceroses evil monkeys terrifying creatures'

>> ## Applying ML Algo : Cosine Similarity between all movies

In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['all'])

In [ ]:
df.shape,df.columns

((20998, 5),
 Index(['id', 'imdb_id', 'poster_path', 'title', 'all'], dtype='object'))

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df.shape

(20998, 5)

In [ ]:
def get_recommendations(title):
    idx = df[df['title']==title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:22]
    movie_indices = [i[0] for i in sim_scores]
    return df.iloc[movie_indices]['id'].values
    # return movie_indices

In [ ]:
type(get_recommendations("PK"))

numpy.ndarray

In [ ]:
df.columns

Index(['id', 'imdb_id', 'poster_path', 'title', 'all'], dtype='object')

In [ ]:
df = df[['id', 'imdb_id','title']]

In [ ]:
df.head()

,id,imdb_id,title
0,8844,tt0113497,Jumanji
1,15602,tt0113228,Grumpier Old Men
2,31357,tt0114885,Waiting to Exhale
3,11862,tt0113041,Father of the Bride Part II
4,949,tt0113277,Heat


In [ ]:
df['recommendations'] = df['title'].apply(get_recommendations)

In [ ]:
# df.to_csv("mrs.csv")

In [ ]:
df['recommendations']

0        [133977, 14237, 10069, 11849, 82690, 59484, 87...
1        [11520, 27472, 18080, 56435, 46732, 76002, 361...
2        [21539, 18169, 448449, 41469, 33644, 13969, 49...
3        [11846, 14603, 96243, 8388, 10678, 10671, 3375...
4        [15542, 9008, 13389, 11524, 6075, 968, 13012, ...
                               ...                        
20993    [4599, 14883, 1833, 1262, 61934, 14160, 42269,...
20994    [14872, 19803, 10118, 8976, 12839, 142499, 340...
20995    [17926, 78133, 21811, 41787, 10012, 173541, 30...
20996    [8005, 26175, 113175, 42657, 10786, 50329, 105...
20997    [219435, 186975, 46773, 418378, 332839, 25051,...
Name: recommendations, Length: 20998, dtype: object

In [ ]:
df['recommendations']  = df['recommendations'].apply(lambda x:list(x))

In [ ]:
df['recommendations'][0]

[133977,
 14237,
 10069,
 11849,
 82690,
 59484,
 879,
 8765,
 56931,
 68591,
 151870,
 45169,
 4442,
 80215,
 45572,
 60760,
 30653,
 11884,
 11551,
 36557,
 257344]

In [ ]:
df.to_csv("mrs2.csv")

In [ ]:
df2 = pd.read_csv("mrs2.csv")

In [ ]:
ast.literal_eval(df2['recommendations'][0])

[133977,
 14237,
 10069,
 11849,
 82690,
 59484,
 879,
 8765,
 56931,
 68591,
 151870,
 45169,
 4442,
 80215,
 45572,
 60760,
 30653,
 11884,
 11551,
 36557,
 257344]

In [ ]:
df.shape

(20998, 4)

In [ ]:
df

,id,imdb_id,title,recommendations
0,8844,tt0113497,Jumanji,"[133977, 14237, 10069, 11849, 82690, 59484, 87..."
1,15602,tt0113228,Grumpier Old Men,"[11520, 27472, 18080, 56435, 46732, 76002, 361..."
2,31357,tt0114885,Waiting to Exhale,"[21539, 18169, 448449, 41469, 33644, 13969, 49..."
3,11862,tt0113041,Father of the Bride Part II,"[11846, 14603, 96243, 8388, 10678, 10671, 3375..."
4,949,tt0113277,Heat,"[15542, 9008, 13389, 11524, 6075, 968, 13012, ..."
...,...,...,...,...
20993,327237,tt3814486,"Blood, Sweat and Tears","[4599, 14883, 1833, 1262, 61934, 14160, 42269,..."
20994,14008,tt0294425,Cadet Kelly,"[14872, 19803, 10118, 8976, 12839, 142499, 340..."
20995,404604,tt5690142,Mom,"[17926, 78133, 21811, 41787, 10012, 173541, 30..."
20996,30840,tt0102797,Robin Hood,"[8005, 26175, 113175, 42657, 10786, 50329, 105..."


In [ ]:
df.to_csv("mrs2.csv")